In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
# Read in data
accounts = pd.read_csv("../Resources/accounts.csv")
ast = pd.read_csv("../Resources/account_state_transitions.csv")
groups = pd.read_csv("../Resources/groups.csv")

## 4) Devise a method of generating a table of all Accounts with their “nominal” expected payments as of any arbitrary timestamp in the past. In other words, we’re looking for the amount the distributor would have collected as of the test date assuming the client had paid the minimum payment instantaneously upon going to a “disabled” state until reaching the unlock price.

In [3]:
groups.head()

,id,name,price_upfront,price_unlock,price_clock_hour,minimum_payment
0,1,Gugun Polar Wireless Product,950,8100,5.654762,71.5
1,2,Tennie JUA Home Mate H1G Lite,1500,6100,8.928571,46.0
2,3,Hong Rath Omnivoltaic Catch (Keycode) Bundle,800,5200,4.761905,44.0
3,4,Monte Omnivoltaic M400,650,6100,3.869048,54.5
4,5,Coats Omnivoltaic M600 Bundle,1100,6100,6.547619,50.0


In [4]:
accounts.head()

,id,group_id,organization_id,registration_date,is_written_off,is_unlocked
0,2,1,10001,2019-11-06 05:49:39.571392,False,True
1,3,1,10001,2019-11-14 14:57:02.571392,False,False
2,4,1,10001,2019-12-11 03:18:53.571392,False,True
3,5,1,10001,2019-10-31 22:36:23.571392,False,False
4,6,1,10001,2019-09-29 13:04:45.571392,False,True


In [5]:
ast.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


## Merge Dataframes

In [6]:
# Merge 'ast' and 'accounts' dataframes
merged_df = pd.merge(left=ast, right=accounts, how='outer', left_on='account_id', right_on='id')

# Limit columns
merged_df = merged_df[['started_when','account_id','to_state','group_id']]
merged_df.head()

,started_when,account_id,to_state,group_id
0,2019-11-06 05:49:39.571392,2,ENABLED,1
1,2019-11-25 22:20:59.150339,2,DISABLED,1
2,2019-11-26 10:22:36.571392,2,ENABLED,1
3,2019-11-29 11:03:32.150339,2,DISABLED,1
4,2019-12-03 10:08:03.571392,2,ENABLED,1


In [8]:
# Merge with 'groups' dataframe
merged_df = pd.merge(left=merged_df, right=groups, how='outer', left_on='group_id', right_on='id')

# Limit columns
merged_df = merged_df.drop(columns=['id','name','price_clock_hour'])
merged_df.head()

,started_when,account_id,to_state,group_id,price_upfront,price_unlock,minimum_payment
0,2019-11-06 05:49:39.571392,2,ENABLED,1,950,8100,71.5
1,2019-11-25 22:20:59.150339,2,DISABLED,1,950,8100,71.5
2,2019-11-26 10:22:36.571392,2,ENABLED,1,950,8100,71.5
3,2019-11-29 11:03:32.150339,2,DISABLED,1,950,8100,71.5
4,2019-12-03 10:08:03.571392,2,ENABLED,1,950,8100,71.5


## Limit by Indicated Timestamp

In [9]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-03-01


In [10]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

# Adjust timestamp to end of day
timestamp_dt = timestamp_dt + timedelta(hours=23,minutes=59,seconds=59)

In [11]:
# Change data type from string to datetime object
merged_df['started_when'] = pd.to_datetime(merged_df['started_when'])

In [12]:
# Create limited dataframe based on indicated timestamp 
merged_limited_df = merged_df.loc[merged_df['started_when'] <= timestamp_dt]

In [14]:
merged_limited_df.head()

,started_when,account_id,to_state,group_id,price_upfront,price_unlock,minimum_payment
0,2019-11-06 05:49:39.571392,2,ENABLED,1,950,8100,71.5
1,2019-11-25 22:20:59.150339,2,DISABLED,1,950,8100,71.5
2,2019-11-26 10:22:36.571392,2,ENABLED,1,950,8100,71.5
3,2019-11-29 11:03:32.150339,2,DISABLED,1,950,8100,71.5
4,2019-12-03 10:08:03.571392,2,ENABLED,1,950,8100,71.5


In [15]:
# Limit dataframe to rows with 'DISABLED' state
disabled_limited_df = merged_limited_df[merged_limited_df['to_state']=='DISABLED']
disabled_limited_df.head()

,started_when,account_id,to_state,group_id,price_upfront,price_unlock,minimum_payment
1,2019-11-25 22:20:59.150339,2,DISABLED,1,950,8100,71.5
3,2019-11-29 11:03:32.150339,2,DISABLED,1,950,8100,71.5
5,2019-12-06 06:44:56.624024,2,DISABLED,1,950,8100,71.5
7,2019-12-12 11:56:56.518760,2,DISABLED,1,950,8100,71.5
9,2019-12-18 11:33:11.202971,2,DISABLED,1,950,8100,71.5


In [16]:
# Group by account and count the occurrences of "DISABLED" state 
disabled_grouped_df = disabled_limited_df.groupby('account_id',as_index=False).agg({
    'to_state':'count',
    'price_upfront':'max',
    'price_unlock':'max',
    'minimum_payment':'max',})

# Rename column
disabled_grouped_df = disabled_grouped_df.rename(columns={'to_state':'count_disabled'})
disabled_grouped_df.head()

,account_id,count_disabled,price_upfront,price_unlock,minimum_payment
0,1,2,950,8100,71.5
1,2,14,950,8100,71.5
2,3,10,950,8100,71.5
3,4,8,950,8100,71.5
4,5,9,950,8100,71.5


In [17]:
# Calculate nominal expected payment
expected_payment = (disabled_grouped_df['count_disabled']*disabled_grouped_df['minimum_payment']) + disabled_grouped_df['price_upfront']

In [19]:
# Make new dataframe with account_id and expected_payment
df = pd.DataFrame({'account_id':disabled_grouped_df['account_id'],
                  f'expected_payment_as_of_{timestamp_str}':expected_payment})

# Preview dataframe
print(f"NOMINAL EXPECTED PAYMENTS BY ACCOUNT AS OF: {timestamp_str[:10]}")
df.head()

NOMINAL EXPECTED PAYMENTS BY ACCOUNT AS OF: 2020-03-01


,account_id,expected_payment_as_of_2020-03-01
0,1,1093.0
1,2,1951.0
2,3,1665.0
3,4,1522.0
4,5,1593.5


In [20]:
# Export to csv
df.to_csv(f"../Output/nominal_expected_payments_as_of_{timestamp_str[:10]}.csv", index=False)